In [1]:
import pandas as pd
import sqlite3 as sql
import warnings
import pyodbc
warnings.simplefilter('ignore')

In [3]:
connection_sales = sql.connect('go_sales_train.sqlite')
connection_crm = sql.connect('go_crm_train.sqlite')
connection_staff = sql.connect('go_staff_train.sqlite')

empty tables

In [ ]:

# Maak verbinding met SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=MSI\\SQLEXPRESS;'  # Gebruik de juiste servernaam uit de screenshot
    'DATABASE=sdm;'  # Gebruik de database 'sdm'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Haal alle tabellen op
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    print(f"🗑️ Legen van tabel: {table_name}")
    cursor.execute(f"DELETE FROM {table_name};")
    conn.commit()

print("✅ Alle tabellen in de database 'sdm' zijn geleegd.")

cursor.close()
conn.close()


🗑️ Legen van tabel: age_group
🗑️ Legen van tabel: country
🗑️ Legen van tabel: course
🗑️ Legen van tabel: order_details
🗑️ Legen van tabel: order_header
🗑️ Legen van tabel: order_method
🗑️ Legen van tabel: product
🗑️ Legen van tabel: product_line
🗑️ Legen van tabel: product_type
🗑️ Legen van tabel: retailer
🗑️ Legen van tabel: retailer_contact
🗑️ Legen van tabel: retailer_headquarters
🗑️ Legen van tabel: retailer_segment
🗑️ Legen van tabel: retailer_site
🗑️ Legen van tabel: retailer_type
🗑️ Legen van tabel: return_reason
🗑️ Legen van tabel: returned_item
🗑️ Legen van tabel: sales_branch
🗑️ Legen van tabel: sales_demographic
🗑️ Legen van tabel: sales_staff
🗑️ Legen van tabel: sales_territory
🗑️ Legen van tabel: satisfaction_type
🗑️ Legen van tabel: training
✅ Alle tabellen in de database 'sdm' zijn geleegd.


importeer data

In [ ]:
import sqlite3 as sql
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=MSI\\SQLEXPRESS;'  # Gebruik de juiste servernaam uit de screenshot
    'DATABASE=sdm;'  # Gebruik de database 'sdm'
    'Trusted_Connection=yes;'
)


cursor_ssms = conn.cursor()

connection_sales = sql.connect('go_sales_train.sqlite')
connection_crm = sql.connect('go_crm_train.sqlite')
connection_staff = sql.connect('go_staff_train.sqlite')

sqlite_connections = {
    "go_sales_train": connection_sales,
    "go_crm_train": connection_crm,
    "go_staff_train": connection_staff
}

csv_files = {
    "inventory_levels_train": "inventory_levels_train.csv",
    "product_forecast_train": "product_forecast_train.csv",
}

for db_name, conn_sqlite in sqlite_connections.items():
    cursor_sqlite = conn_sqlite.cursor()
    cursor_sqlite.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor_sqlite.fetchall()
    
    for table in tables:
        table_name = table[0]
        print(f"Importeren van {table_name} uit {db_name}")
        
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn_sqlite)

        if not df.empty:  
            sqlite_columns = df.columns.tolist()

            cursor_ssms.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'")
            sql_columns = {row[0]: row[1] for row in cursor_ssms.fetchall()}

            common_columns = [col for col in sqlite_columns if col in sql_columns]
            if not common_columns:
                print(f"Geen overeenkomende kolommen voor {table_name} tussen SQLite en SQL Server.")
                continue

            for col in common_columns:
                if sql_columns[col] == 'float':
                    df[col] = df[col].apply(lambda x: x if is_valid_float(x) else None)

            columns_str = ", ".join(common_columns)
            placeholders_str = ", ".join(["?"] * len(common_columns))

            for index, row in df[common_columns].iterrows():
                try:
                    print(f"Probeer in te voegen: {tuple(row)}")  
                    
                    sql_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders_str})"
                    cursor_ssms.execute(sql_query, tuple(row))
                except Exception as e:
                    print(f"Fout bij het invoegen van rij {index} in tabel {table_name}: {e}")
                    print(f"Probleemrij: {tuple(row)}")  
                    continue

            conn.commit()

for table_name, file_path in csv_files.items():
    
    df = pd.read_csv(file_path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  

    for index, row in df.iterrows():
        try:
            columns = ", ".join(df.columns)
            placeholders = ", ".join(["?"] * len(df.columns))
            sql_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            print(f"Probeer in te voegen uit CSV: {tuple(row)}") 
            cursor_ssms.execute(sql_query, tuple(row))
        except Exception as e:
            print(f"Fout bij het invoegen van rij {index} uit CSV-bestand {table_name}: {e}")
            print(f"Probleemrij: {tuple(row)}")  
            continue

    conn.commit()

print("Data import voltooid.")

cursor_ssms.close()
conn.close()
for conn in sqlite_connections.values():
    conn.close()


Importeren van country uit go_sales_train
Geen overeenkomende kolommen voor country tussen SQLite en SQL Server.
Importeren van order_details uit go_sales_train
Geen overeenkomende kolommen voor order_details tussen SQLite en SQL Server.
Importeren van order_header uit go_sales_train
Geen overeenkomende kolommen voor order_header tussen SQLite en SQL Server.
Importeren van order_method uit go_sales_train
Geen overeenkomende kolommen voor order_method tussen SQLite en SQL Server.
Importeren van product_line uit go_sales_train
Geen overeenkomende kolommen voor product_line tussen SQLite en SQL Server.
Importeren van product_type uit go_sales_train
Geen overeenkomende kolommen voor product_type tussen SQLite en SQL Server.
Importeren van product uit go_sales_train
Geen overeenkomende kolommen voor product tussen SQLite en SQL Server.
Importeren van return_reason uit go_sales_train
Geen overeenkomende kolommen voor return_reason tussen SQLite en SQL Server.
Importeren van returned_item uit